In [62]:
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
import time
from tqdm import tqdm
from IPython.display import Image
import regex as re
from IPython.core.display import display,HTML
from joblib import Parallel, delayed

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
PATH=ChromeDriverManager().install()

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.46M/6.46M [00:00<00:00, 37.2MB/s]


In [ ]:
# EL REFUGIO

In [3]:
url = 'https://elrefugio.org/adopta/perros'
driver=webdriver.Chrome(PATH) 
driver.get(url)
time.sleep(3)

muchos=driver.find_elements(By.CLASS_NAME, 'item')
uno = [e.find_element(By.TAG_NAME, 'a') for e in muchos]
links = [e.get_attribute('href') for e in uno]

imagen=[]
nombres= []
carac= []
protectora=[]
for e in tqdm(links):
    protectora.append(1)
    driver.get(e)
    box = driver.find_element(By.CLASS_NAME, 'boxSlideFotos')
    img = box.find_element(By.TAG_NAME, 'img')
    imagen.append(img.get_attribute('src'))
    desc = driver.find_element(By.CLASS_NAME, 'txtDoble')
    nombres.append(desc.find_element(By.TAG_NAME, 'h1').text)
    todo = desc.find_elements(By.TAG_NAME, 'li')
    for i in todo:
        i = i.text
        i = i.split(':')
        i.pop(0)
        carac += i
        
driver.quit()

n = 6
caracter = [carac[i:i + n] for i in range(0, len(carac), n)]

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [00:38<00:00,  1.86it/s]


In [5]:
colnames= ['SEXO', 'EDAD', 'NACIMIENTO', 'RAZA', 'TAMAÑO', 'CARACTER']
perros = pd.DataFrame(caracter, columns=colnames)

perros['NOMBRE']= nombres
perros['PROTECTORA']= protectora

sexo= []
for e in perros.SEXO:
    e = e.lower()
    e= e.strip()
    sexo.append(e)
perros['SEXO']= sexo

edad= []
for e in perros.EDAD:
    e = e.lower()
    e= e.strip()
    edad.append((re.findall('[a-z]+', e)[0]))
perros['EDAD']= edad

raza= []
for e in perros.RAZA:
    e = e.lower()
    e= e.strip()
    raza.append(e)
perros['RAZA']= raza

tam= []
for e in perros.TAMAÑO:
    e = e.lower()
    e= e.strip()
    tam.append(e)
perros['TAMAÑO']= tam

car= []
for e in perros.CARACTER:
    e = e.lower()
    e= e.strip()
    car.append(e)
perros['CARACTER']= car

nom= []
for e in perros.NOMBRE:
    e = e.capitalize()
    e= e.strip()
    nom.append(e)
perros['NOMBRE']= nom

nac= []
for e in perros.NACIMIENTO:
    e= e.strip()
    nac.append(e)
perros['NACIMIENTO']= nac
# perros['NACIMIENTO'] = perros['NACIMIENTO'].astype('datetime64[ns]')



images1 = imagen
perros['FOTO'] = images1

# convert your links to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

pd.set_option('display.max_colwidth', None)

image_cols = ['FOTO']  #<- define which columns will be used to convert to html

# Create the dictionariy to be passed as formatters
format_dict = {}
for e in image_cols:
    format_dict[e] = path_to_image_html


#display(HTML(perros.to_html(escape=False ,formatters=format_dict)))




In [6]:
images1 = imagen

perros['FOTO'] = images1


# convert your links to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

pd.set_option('display.max_colwidth', None)

image_cols = ['FOTO']  #<- define which columns will be used to convert to html

# Create the dictionariy to be passed as formatters
format_dict = {}
for e in image_cols:
    format_dict[e] = path_to_image_html
#display(HTML(perros.to_html(escape=False ,formatters=format_dict)))


perros.RAZA.replace({'galga': 'galgo', 'mestiza': 'mestizo'})
perros.TAMAÑO.replace({'medio': 'mediano', 'mediana': 'mediano'})
perros.CARACTER.replace({'muy cariñosa': 'cariñosa', 'timida': 'tímida'}).head()

0      cariñoso
1        tímida
2      cariñoso
3        tímido
4     tranquilo
        ...    
67       tímido
68       activa
69       tímido
70     cariñoso
71    simpático
Name: CARACTER, Length: 72, dtype: object

In [7]:
colnames= ('ID', 'NOMBRE', 'UBICACION', 'CONTACTO')
protectora = pd.DataFrame(columns=colnames)
elrefugio= {'ID': 1, 'NOMBRE': 'El Refugio', 'UBICACION': 'MADRID', 'CONTACTO': '(+34) 91 730 3680'}
protectora = protectora.append(elrefugio, ignore_index = True)

In [ ]:
# ACUNR

In [77]:
url = 'https://www.acunr.es/perros/'
driver=webdriver.Chrome(PATH) 
driver.get(url)

links= []

for e in tqdm(range(5)):
    muchos = driver.find_element(By.CLASS_NAME, 'col-lg-12')
    uno = muchos.find_elements(By.TAG_NAME, 'a')
    links += (e.get_attribute('href') for e in uno)
    otra = driver.find_element(By.XPATH, '//*[@id="main"]/ul[1]/li[8]/a')
    otra.click()
'''
ESTO NO LO PONGO DENTRO DEL BUCLE PORQUE ES LA ULTIMA PAG
SI LO PONGO DENTRO DEL BUCLE, INTENTA DAR A CLICK Y ME DA ERROR
PORQUE NO HAY OTRA PAG
'''
muchos = driver.find_element(By.CLASS_NAME, 'col-lg-12')
uno = muchos.find_elements(By.TAG_NAME, 'a')
links += [e.get_attribute('href') for e in uno]


links2= []
for e in range(len(links)):
    if e % 2:
        links2.append(links[e])
    else:
        pass
    
    

imagen=[]
nombres= []
carac= []
protectora=[]
for e in tqdm(links2):
    try:
        protectora.append(2) 
        driver.get(e)
        time.sleep(0.2)
        nombres.append(driver.find_element(By.TAG_NAME, 'h2').text)
        box = driver.find_element(By.CLASS_NAME, 'col-lg-12')
        img= box.find_element(By.TAG_NAME, 'img')
        imagen.append(img.get_attribute('src'))
        desc = box.find_elements(By.CLASS_NAME, 'card-text')
        for i in desc:
            i = i.text
            i = i.split(':')
            i.pop(0)
            carac += i
    
    except:
        print(e)

driver.quit()

100%|██████████████████████████████████████████████████████████████████████████████████| 52/52 [02:54<00:00,  3.35s/it]


In [75]:
'''
ESTE CODIGO TARDA MÁS QUE SI VA UNO POR UNO, 
ASÍ QUE NO LO VOY A USAR


url = 'https://www.acunr.es/perros/'
driver=webdriver.Chrome(PATH) 
driver.get(url)

links= []

for e in tqdm(range(5)):
    perros = driver.find_element(By.CLASS_NAME, 'col-lg-12')
    perro = perros.find_elements(By.TAG_NAME, 'a')
    links += (e.get_attribute('href') for e in perro)
    otra = driver.find_element(By.XPATH, '//*[@id="main"]/ul[1]/li[8]/a')
    otra.click()
perros = driver.find_element(By.CLASS_NAME, 'col-lg-12')
perro = perros.find_elements(By.TAG_NAME, 'a')
links += [e.get_attribute('href') for e in perro]


links2= []
for e in range(len(links)):
    if e % 2:
        links2.append(links[e])
    else:
        pass


def extraer(url):
    imagen=[]
    nombres= []
    carac= []
    protectora=[]
    try:
        protectora.append(2)
        driver=webdriver.Chrome(PATH) 
        driver.get(e)
        nombres.append(driver.find_element(By.TAG_NAME, 'h2').text)
        box = driver.find_element(By.CLASS_NAME, 'col-lg-12')
        img= box.find_element(By.TAG_NAME, 'img')
        imagen.append(img.get_attribute('src'))
        desc = box.find_elements(By.CLASS_NAME, 'card-text')
        for i in desc:
            i = i.text
            i = i.split(':')
            i.pop(0)
            carac += i

    except:
        print(e)

    driver.quit()
    
    return 'all right'

all_right=Parallel(n_jobs=3, verbose=True)(delayed(extraer)(url) for url in tqdm(links2))
'''

In [79]:
perros

<selenium.webdriver.remote.webelement.WebElement (session="b38c6d3afe050dc0177127f15563be96", element="a6c8adda-c719-466a-8fb8-69b70c549d3b")>